# Imports

In [107]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
# global imports
import os
from google.colab import drive


In [109]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import json
import re
import nltk
import string

In [110]:
! pip install contractions
!pip install -U spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 844.2 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [111]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [112]:
import spacy
# import ktrain
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import re
from sklearn.feature_extraction.text import CountVectorizer
#from ktrain.text import StandardTextPreprocessor
import contractions
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string
# getting stop_words from nltk and initializing a lemmatizer
stop_words = set(nltk.corpus.stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()
from sklearn.cluster import KMeans
import time
import pickle
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


# Data Cleaning

In [113]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
# import cupy as cp
import pandas as pd
import xml.etree.ElementTree as ET
from google.colab import drive
from sklearn.compose import make_column_transformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [114]:
class ABSADataset(Dataset):
    def __init__(self, texts, aspects, labels, tokenizer, max_length=128):
        self.texts = texts
        self.aspects = aspects
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        aspect = self.aspects[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(text, aspect,
                                  return_tensors='pt',
                                  max_length=self.max_length,
                                  truncation=True,
                                  padding='max_length')

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label)
        }

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Convert to lowercase
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into string
    clean_text = ' '.join(filtered_tokens)

    return clean_text


def clean_aspect(aspect):
    # Convert to lowercase
    aspect = aspect.lower()

    # Remove punctuation
    aspect = aspect.translate(str.maketrans('', '', string.punctuation))

    return aspect

In [115]:
# load the dataset with labels

dataset_path = '/content/drive/MyDrive/Sem4/NLP/Final project/'
dataset_path = dataset_path + 'Restaurants_Train_v2.xml'
dataset = []

f = ET.parse(dataset_path)
for i in f.getroot():
  for j in i.find("aspectCategories"):
    dataset.append({
        "input_text": clean_text(i.find("text").text),
        "aspect": clean_aspect(j.attrib['category']),
        "polarity": j.attrib['polarity']
    })

bgw_vectorizer = CountVectorizer(binary=False, stop_words='english', ngram_range=(1,4))
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [116]:
df = pd.DataFrame(train_dataset)

In [117]:
df.head()

,input_text,aspect,polarity
0,decor nice casual fine dining,ambience,conflict
1,find returning quite often,anecdotesmiscellaneous,positive
2,seating trying get maximum amount people resta...,ambience,negative
3,like took leftover chicken poured oil sprinkle...,food,negative
4,finest latin women youll see life hang jimmys,ambience,positive


In [118]:
df['input_text'][4]

'finest latin women youll see life hang jimmys'

# Getting aspect nouns



In [ ]:
# Load spaCy model
nlp = spacy.load("en_core_web_lg")


# Define a function to extract nouns (aspects) from a review
def extract_nouns_from_review(review, nlp):
    # Process the review using spaCy pipeline
    doc = nlp(review)
    # Extract nouns from the review
    nouns = [token.text for token in doc if token.pos_ == "NOUN"]
    return list(set(nouns))

# Extract nouns (aspects) for each review and add them as a new column to the DataFrame
df['extracted_nouns'] = df['input_text'].apply(lambda review: extract_nouns_from_review(review, nlp))

In [ ]:
df.head()

,input_text,aspect,polarity,extracted_nouns
0,decor nice casual fine dining,ambience,conflict,"[dining, decor]"
1,find returning quite often,anecdotesmiscellaneous,positive,[]
2,seating trying get maximum amount people resta...,ambience,negative,"[people, neighbor, amount, seating]"
3,like took leftover chicken poured oil sprinkle...,food,negative,"[pepper, sauce, oil, chicken, powder, red]"
4,finest latin women youll see life hang jimmys,ambience,positive,"[women, life]"


In [119]:
df.shape

(2970, 3)

In [120]:
df = df[~(df['input_text']=='')]

In [121]:
df.shape

(2967, 3)

In [122]:
df.to_csv(dataset_path+'semeval_cleaned.csv', index=False)

In [123]:
df=pd.read_csv(dataset_path+'semeval_cleaned.csv')

In [125]:
df.shape

(2967, 3)

# Getting aspect categories

In [126]:
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Load spaCy model with GloVe vectors
#!python -m spacy download en_core_web_lg #run this for every new session
nlp = spacy.load("en_core_web_lg")

# Define aspects and their corresponding words
aspects = {
    'food': 'food delicious tasty delicious flavor meal cuisine chef portion seasoned authentic',
    'service': 'service staff friendly helpful attentive prompt',
    'ambience': 'ambience ambiance atmosphere decor decoration cozy elegant modern chic view clean dirty dusty',
    'price': 'price value affordable expensive budget cost cheap',
    # 'anecdotesmiscellaneous': 'location parking cleanliness presentation variety'

}


# Define a function to compute aspect embeddings
def compute_aspect_embeddings(aspects, nlp):
    aspect_embeddings = {}
    for aspect, words in aspects.items():
        # Tokenize and average the word vectors for each aspect
        tokens = nlp(words)
        aspect_vectors = [token.vector for token in tokens if token.has_vector]
        aspect_embeddings[aspect] = np.mean(aspect_vectors, axis=0)
    return aspect_embeddings

# Define a function to compute review embeddings
def compute_review_embedding(review, nlp):
    # Process the review using spaCy pipeline
    doc = nlp(review)
    sentence_embeddings = []
    for sent in doc.sents:
        # Compute sentence embedding by averaging word vectors
        sent_vector = np.mean([word.vector for word in sent if word.has_vector], axis=0)
        sentence_embeddings.append(sent_vector)
    # Replace new words with zeros and compute review embedding
    review_embedding = np.mean(sentence_embeddings, axis=0)
    return np.nan_to_num(review_embedding)


def compute_sentence_embedding(sentence, nlp):
    # Process the sentence using spaCy pipeline
    doc = nlp(sentence)
    # Compute sentence embedding by averaging word vectors
    sent_vector = np.mean([word.vector for word in doc if word.has_vector], axis=0)
    # Replace NaN values with zeros
    return np.nan_to_num(sent_vector)


# Compute semantic similarity between review sentences and aspect embeddings
def compute_semantic_similarity(review_sentences, aspect_embeddings):
    aspect_names = list(aspect_embeddings.keys())
    similarity_scores = []
    i=0
    for sentence in review_sentences:
        sentence_vector = compute_review_embedding(sentence, nlp)
        sentence_vector = sentence_vector.reshape(1, -1)  # Reshape to match cosine_similarity function input
        sentence_similarity = []
        for aspect_name, aspect_vector in aspect_embeddings.items():
            aspect_vector = aspect_vector.reshape(1, -1)  # Reshape to match cosine_similarity function input
            similarity = cosine_similarity(sentence_vector, aspect_vector)[0][0]
            sentence_similarity.append(similarity)
        similarity_scores.append({aspect_name: score for aspect_name, score in zip(aspect_names, sentence_similarity)})
        print(i)
    return similarity_scores

In [127]:
df2 = df.copy()
df2['input_text_len'] = df2['input_text'].apply(lambda x: len(nltk.word_tokenize(x)))

In [129]:
df2 = df2[df2['input_text_len']>1]

In [131]:
df2.shape

(2904, 4)

In [ ]:
# review_sentences = df['input_text'][0:100].to_list() --working fine

# review_sentences = df['input_text']

review_sentences = df2['input_text']

#for each review in 'cleaned_text' and get a list of aspects

# Compute aspect embeddings
aspect_embeddings = compute_aspect_embeddings(aspects, nlp)

# Compute semantic similarity
semantic_similarity_scores = compute_semantic_similarity(review_sentences, aspect_embeddings)

# Choose the aspect with the highest similarity score for each sentence
chosen_aspects = [max(scores, key=scores.get) for scores in semantic_similarity_scores]

In [106]:
df['input_text'][426]

'bravissimo'

In [132]:
df.shape

(2967, 3)

In [81]:
# Output the chosen aspects for each sentence
for sentence, aspect in zip(review_sentences, chosen_aspects):
    print("Sentence:", sentence)
    print("Chosen Aspect:", aspect)
    print()

Sentence: decor nice casual fine dining
Chosen Aspect: ambience

Sentence: seating trying get maximum amount people restaurant nice neighbor dine
Chosen Aspect: service

Sentence: like took leftover chicken poured oil sprinkled pepper powder sauce translucent red
Chosen Aspect: food

Sentence: finest latin women youll see life hang jimmys
Chosen Aspect: ambience

Sentence: ive bunch times service always outstanding
Chosen Aspect: service

Sentence: food outstanding little perks great
Chosen Aspect: food

Sentence: makhani ok korma bland
Chosen Aspect: food

Sentence: make reservations expect delayed 1520 minutes hosting staff difficulty seating guests arrived reservation probably lot walk ins close time square
Chosen Aspect: service

Sentence: need kick sushi pretty good place consistent
Chosen Aspect: food

Sentence: keep capex minimum put cash bagels among best city
Chosen Aspect: price

Sentence: wont break bank also wouldnt come back food
Chosen Aspect: price

Sentence: staff atten

In [ ]:
df2 = df[0:100]

In [133]:
df2['aspect_predicted'] = chosen_aspects

In [134]:
df2.head()

,input_text,aspect,polarity,input_text_len,aspect_predicted
0,decor nice casual fine dining,ambience,conflict,5,ambience
1,find returning quite often,anecdotesmiscellaneous,positive,4,service
2,seating trying get maximum amount people resta...,ambience,negative,10,service
3,like took leftover chicken poured oil sprinkle...,food,negative,12,food
4,finest latin women youll see life hang jimmys,ambience,positive,8,ambience


# Accuracy check

In [135]:
df2 = df2[df['aspect'] != "anecdotesmiscellaneous"]

<ipython-input-135-9a1b6a97aeb8>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df2[df['aspect'] != "anecdotesmiscellaneous"]


In [136]:
cnt = (df2['aspect'] == df2['aspect_predicted']).sum()

In [137]:
cnt

1318

In [138]:
df2.shape

(2033, 5)

In [139]:
1318/2033

0.6483030004918839

# 65% accuracy